In [1]:
import os, sys
import warnings
warnings.filterwarnings('ignore')
from cellProcessing_single_WS import *

dir_root = '/nrs/ahrens/Yu/SPIM/active_dataset/20141209_1_1_cy74_GAD_1_1_simpleGU_20141209_154521/raw'
save_root = '/nrs/ahrens/Ziqiang/Pipeline_test/Yu_TK_pipeline/'
dask_tmp = '/groups/ahrens/home/weiz/thinclient_drives/dask-worker-space'
memory_limit = 0 # unlimited

if not os.path.exists(save_root):
    os.makedirs(save_root)

nsplit = (4, 8)
baseline_percentile = 20  
baseline_window = 400   # number of frames
cameraNoiseMat = '/nrs/ahrens/ahrenslab/Ziqiang/gainMat/gainMat20180208'

if not os.path.exists(f'{save_root}/detrend_data.zarr'):
    preprocessing(dir_root, save_root, cameraNoiseMat=cameraNoiseMat, window=baseline_window, 
                  percentile=baseline_percentile, nsplit=nsplit, dask_tmp=dask_tmp, memory_limit=memory_limit, is_bz2=True)

In [2]:
cluster, client = fdask.setup_workers(is_local=True, dask_tmp=dask_tmp, memory_limit=memory_limit)
print_client_links(cluster)

Scheduler: inproc://10.150.100.212/344994/1
Dashboard link: http://10.150.100.212/344994/1:8787/status


In [3]:
Y = da.from_zarr(f'{save_root}/motion_corrected_data.zarr')
Y_d = da.from_zarr(f'{save_root}/detrend_data.zarr')

In [12]:
fdask.terminate_workers(cluster, client)

In [6]:
Y.shape

(35, 1084, 2048, 5150)

In [10]:
2048/8

256.0

In [ ]:
Y = da.from_zarr(f'{save_root}/motion_corrected_data.zarr')
Y_d = da.from_zarr(f'{save_root}/detrend_data.zarr')
Y_b = Y - Y_d
Y_b_max_mask = Y_b.max(axis=-1, keepdims=True)>2
Y_b_min_mask = Y_b.min(axis=-1, keepdims=True)>1
mask = Y_b_max_mask & Y_b_min_mask
mask.to_zarr(f'{save_root}/mask_map.zarr', overwrite=True)


Y_d = da.from_zarr(f'{save_root}/detrend_data.zarr')
Y_d_max = Y_d.max(axis=-1, keepdims=True)
print('Save average data ---')
Y_d_max.to_zarr(f'{save_root}/Y_max.zarr', overwrite=True)  


Y_d = zarr.open(f'{save_root}/Y_max.zarr', 'r')
mask = zarr.open(f'{save_root}/mask_map.zarr', 'r')
for n, n_ave_ in enumerate(Y_d):
    _ = n_ave_.squeeze().copy()
    _[~mask[n].squeeze()] = 0
    plt.imshow(_, vmax=np.percentile(_, 99))
    plt.title(n)
    plt.show()